In [1]:
!pip install kaggle

استدعاء DATASET "MSVD_CLIPS"

In [2]:
!kaggle datasets download -d sarthakjain004/msvd-clips

Dataset URL: https://www.kaggle.com/datasets/sarthakjain004/msvd-clips
License(s): unknown
100% 1.71G/1.71G [01:17<00:00, 30.2MB/s]
100% 1.71G/1.71G [01:17<00:00, 23.6MB/s]


استدعاء داتا التوصيفات

In [3]:
!kaggle datasets download -d vtrnanh/msvd-dataset-corpus

Dataset URL: https://www.kaggle.com/datasets/vtrnanh/msvd-dataset-corpus
License(s): unknown
 65% 2.00M/3.09M [00:00<00:00, 2.80MB/s]
100% 3.09M/3.09M [00:00<00:00, 3.48MB/s]


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


فك الضغط

In [5]:
import zipfile
dataset='/content/msvd-clips.zip'
# استخراج الملفات المضغوطة
with zipfile.ZipFile('/content/msvd-clips.zip', 'r') as zip_ref:
    zip_ref.extractall('msvd-clips')
    print("The dataset is extructed")

The dataset is extructed


In [6]:
import zipfile
# استخراج الملفات المضغوطة
with zipfile.ZipFile('/content/msvd-dataset-corpus.zip', 'r') as zip_ref:
    zip_ref.extractall('MSVD_DATASET_CORPUS')
    print("The dataset is extructed")

The dataset is extructed


تقطيع ل frame  , feature , زادخالها الى نموذج   cnn(resnet152)

In [7]:
import numpy as np
import cv2
import os
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.models import Model
from google.colab import drive
import shutil

# ربط Google Drive بجلسة Colab
drive.mount('/content/drive')

# تعديل config مباشرة هنا بدلاً من ملف خارجي
class config:
    test_path = '/content/msvd-clips'  # مسار مقاطع الفيديو على Google Drive
    test_path1 = '/content/drive/My Drive/dataset'  # مسار الميزات والإطارات المستخرجة على Google Drive

def video_to_frames(video):
    video_id = video.split(".")[0]  # اسم مقطع الفيديو
    path = os.path.join(config.test_path1, 'temporary_images', video_id)  # المسار لمجلد الفريمات لهذا المقطع

    # إذا كانت الفريمات موجودة، قم بإرجاعها بدون إعادة الاستخراج
    if os.path.exists(path) and len(os.listdir(path)) > 0:
        image_list = [os.path.join(path, img) for img in sorted(os.listdir(path)) if img.endswith('.jpg')]
        return image_list

    # إذا لم تكن الفريمات موجودة، قم باستخراجها
    os.makedirs(path, exist_ok=True)
    video_path = os.path.join(config.test_path, 'YouTubeClips', video)
    count = 0
    image_list = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        cv2.imwrite(os.path.join(path, f'frame{count}.jpg'), frame)  # حفظ الفريم في مجلد المقطع
        image_list.append(os.path.join(path, f'frame{count}.jpg'))
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return image_list

def model_cnn_load():
    model = ResNet152(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final

def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img

def extract_features(video, model):
    video_id = video.split(".")[0]
    print(video_id)
    print(f'Processing video {video}')

    image_list = video_to_frames(video)
    samples = np.round(np.linspace(0, len(image_list) - 1, 80))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    return img_feats

def extract_feats_pretrained_cnn(limit=30):
    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(config.test_path1, 'feat')):
        os.makedirs(os.path.join(config.test_path1, 'feat'))

    video_list = os.listdir(os.path.join(config.test_path, 'YouTubeClips'))

    if '.ipynb_checkpoints' in video_list:
        video_list.remove('.ipynb_checkpoints')

    # تحديد عدد مقاطع الفيديو المراد استخراج الميزات منها
    video_list = video_list[:limit]

    for video in video_list:
        video_id = video.split(".")[0]
        outfile = os.path.join(config.test_path1, 'feat', video_id + '.npy')

        # إذا كانت الميزات موجودة، تخطي هذا الفيديو
        if os.path.exists(outfile):
            print(f"Features for video {video} already exist. Skipping extraction.")
            continue

        # إذا لم تكن الميزات موجودة، استخراج الميزات والفريمات
        img_feats = extract_features(video, model)
        np.save(outfile, img_feats)
        print(f"Features for video {video} have been extracted and saved.")

# لتشغيل الكود مباشرة مع تحديد الحد الأقصى لعدد مقاطع الفيديو
extract_feats_pretrained_cnn(limit=30)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

هون لشوف قدي عدد الفيديوهات يلي استخرجت منها

In [ ]:
import os

# تعديل config مباشرة هنا بدلاً من ملف خارجي
class config:
    test_path1 = '/content/drive/My Drive/dataset'  # مسار الميزات والإطارات المستخرجة على Google Drive

def count_extracted_features_and_frames():
    # مسار مجلد الميزات
    feat_path = os.path.join(config.test_path1, 'feat')
    # مسار مجلد الفريمات المؤقتة
    temporary_images_path = os.path.join(config.test_path1, 'temporary_images')

    # التحقق من وجود المجلدات
    if not os.path.exists(feat_path):
        print("The features directory does not exist.")
        return
    if not os.path.exists(temporary_images_path):
        print("The temporary images directory does not exist.")
        return

    # عد ملفات الميزات
    feat_files = [f for f in os.listdir(feat_path) if f.endswith('.npy')]
    num_feat_files = len(feat_files)

    # عد مجلدات الفريمات
    frame_dirs = [d for d in os.listdir(temporary_images_path) if os.path.isdir(os.path.join(temporary_images_path, d))]
    num_frame_dirs = len(frame_dirs)

    print(f"Number of videos with extracted features: {num_feat_files}")
    print(f"Number of videos with extracted frames: {num_frame_dirs}")

# استدعاء الدالة لحساب عدد مقاطع الفيديو
count_extracted_features_and_frames()


Number of videos with extracted features: 48
Number of videos with extracted frames: 48


In [ ]:
import os

# تعديل config مباشرة هنا بدلاً من ملف خارجي
class config:
    test_path1 = '/content/drive/My Drive/dataset'  # مسار الميزات والإطارات المستخرجة على Google Drive

def check_matching_video_names():
    # مسار مجلد الميزات
    feat_path = os.path.join(config.test_path1, 'feat')
    # مسار مجلد الفريمات المؤقتة
    temporary_images_path = os.path.join(config.test_path1, 'temporary_images')

    # التحقق من وجود المجلدات
    if not os.path.exists(feat_path) or not os.path.exists(temporary_images_path):
        print("One or both directories do not exist.")
        return

    # استخراج قائمة بأسماء ملفات الميزات (بدون امتداد .npy)
    feat_video_names = [f.split('.')[0] for f in os.listdir(feat_path) if f.endswith('.npy')]

    # استخراج قائمة بأسماء مجلدات الفريمات
    frame_video_names = [d for d in os.listdir(temporary_images_path) if os.path.isdir(os.path.join(temporary_images_path, d))]

    # الفيديوهات التي تم استخراج الفريمات منها ولم يتم استخراج الميزات
    unmatched_videos = list(set(frame_video_names) - set(feat_video_names))

    # الفيديوهات التي تم استخراج الميزات والفريمات منها
    matched_videos = list(set(feat_video_names) & set(frame_video_names))

    print(f"Number of videos with extracted features: {len(feat_video_names)}")
    print(f"Number of videos with extracted frames: {len(frame_video_names)}")
    print(f"Number of videos with matching features and frames: {len(matched_videos)}")
    print(f"Number of videos with frames only (features missing): {len(unmatched_videos)}")
    print(f"Videos with frames only: {unmatched_videos}")

# استدعاء الدالة لفحص تطابق أسماء الفيديوهات
check_matching_video_names()


Number of videos with extracted features: 47
Number of videos with extracted frames: 48
Number of videos with matching features and frames: 47
Number of videos with frames only (features missing): 1
Videos with frames only: ['3OPQqH3YlHA_50_54']


لاستخرج الفيديو يلي ما كنت مستخرجته قبل

In [ ]:
import numpy as np
import cv2
import os
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.models import Model
from google.colab import drive
import shutil

# ربط Google Drive بجلسة Colab
drive.mount('/content/drive')

# تعديل config مباشرة هنا بدلاً من ملف خارجي
class config:
    test_path = '/content/msvd-clips'  # مسار مقاطع الفيديو على Google Drive
    test_path1 = '/content/drive/My Drive/dataset'  # مسار الميزات والإطارات المستخرجة على Google Drive

def video_to_frames(video):
    video_id = video.split(".")[0]  # اسم مقطع الفيديو
    path = os.path.join(config.test_path1, 'temporary_images', video_id)  # المسار لمجلد الفريمات لهذا المقطع

    # إذا كانت الفريمات موجودة، قم بإرجاعها بدون إعادة الاستخراج
    if os.path.exists(path) and len(os.listdir(path)) > 0:
        image_list = [os.path.join(path, img) for img in sorted(os.listdir(path)) if img.endswith('.jpg')]
        return image_list

    # إذا لم تكن الفريمات موجودة، قم باستخراجها
    os.makedirs(path, exist_ok=True)
    video_path = os.path.join(config.test_path, 'YouTubeClips', video)
    count = 0
    image_list = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        cv2.imwrite(os.path.join(path, f'frame{count}.jpg'), frame)  # حفظ الفريم في مجلد المقطع
        image_list.append(os.path.join(path, f'frame{count}.jpg'))
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return image_list

def model_cnn_load():
    model = ResNet152(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final

def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img

def extract_features(video, model):
    video_id = video.split(".")[0]
    print(video_id)
    print(f'Processing video {video}')

    image_list = video_to_frames(video)
    samples = np.round(np.linspace(0, len(image_list) - 1, 80))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    return img_feats

def extract_feats_single_video(video):
    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(config.test_path1, 'feat')):
        os.makedirs(os.path.join(config.test_path1, 'feat'))

    video_id = video.split(".")[0]
    outfile = os.path.join(config.test_path1, 'feat', video_id + '.npy')

    # إذا كانت الميزات موجودة، تخطي هذا الفيديو
    if os.path.exists(outfile):
        print(f"Features for video {video} already exist. Skipping extraction.")
        return

    # إذا لم تكن الميزات موجودة، استخراج الميزات والفريمات
    img_feats = extract_features(video, model)
    np.save(outfile, img_feats)
    print(f"Features for video {video} have been extracted and saved.")

# استدعاء الدالة لاستخراج الميزات من المقطع الفيديو المفقود
extract_feats_single_video('3OPQqH3YlHA_50_54.mp4')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
242900224/242900224 [==============================] - 2s 0us/step
Model loaded
3OPQqH3YlHA_50_54
Processing video 3OPQqH3YlHA_50_54.mp4
1/1 [==============================] - 46s 46s/step
Features for video 3OPQqH3YlHA_50_54.mp4 have been extracted and saved.


عدد الفيديوهات مرة تانية

In [ ]:
import os

# تعديل config مباشرة هنا بدلاً من ملف خارجي
class config:
    test_path1 = '/content/drive/My Drive/dataset'  # مسار الميزات والإطارات المستخرجة على Google Drive

def count_extracted_features_and_frames():
    # مسار مجلد الميزات
    feat_path = os.path.join(config.test_path1, 'feat')
    # مسار مجلد الفريمات المؤقتة
    temporary_images_path = os.path.join(config.test_path1, 'temporary_images')

    # التحقق من وجود المجلدات
    if not os.path.exists(feat_path):
        print("The features directory does not exist.")
        return
    if not os.path.exists(temporary_images_path):
        print("The temporary images directory does not exist.")
        return

    # عد ملفات الميزات
    feat_files = [f for f in os.listdir(feat_path) if f.endswith('.npy')]
    num_feat_files = len(feat_files)

    # عد مجلدات الفريمات
    frame_dirs = [d for d in os.listdir(temporary_images_path) if os.path.isdir(os.path.join(temporary_images_path, d))]
    num_frame_dirs = len(frame_dirs)

    print(f"Number of videos with extracted features: {num_feat_files}")
    print(f"Number of videos with extracted frames: {num_frame_dirs}")

# استدعاء الدالة لحساب عدد مقاطع الفيديو
count_extracted_features_and_frames()


Number of videos with extracted features: 48
Number of videos with extracted frames: 48


معرفة اسماء المقاطع يلي استخرجت منها للميزات والفريمات

In [ ]:
import os

def extract_video_names_from_frames():
    # مسار مجلد الفريمات المؤقتة
    frames_folder_path = '/content/drive/My Drive/dataset/temporary_images'

    # استخراج أسماء مقاطع الفيديو من اسماء الملفات داخل المجلد
    video_names = [filename.split('_')[0] for filename in os.listdir(frames_folder_path)]

    return video_names

# استدعاء الدالة لاستخراج أسماء مقاطع الفيديو
video_names = extract_video_names_from_frames()

# طباعة أسماء مقاطع الفيديو
print("Video names extracted from frames:")
print(video_names)


Video names extracted from frames:
['q5ZRMvjzhXQ', 'KqKUXRHHkCc', 's-XjRDsYuzU', 'lexLAjh8fPA', 'JYNzY9jmsQQ', 'HZ-BuDDmvVk', 'AzMrltkXGJk', '5E66Gk3V1Bc', '6FmN22YcVVE', 'ACOmKiJDkA4', '6gQu8PWhFoQ', 'ptHkvYrH9fY', 'ao-9B8IV9', 'R24RktgUi-s', '', 'HJHPB', 'QsC8', 'r4qv', 'Pr5LQq42l4M', 'n', 'Qjbcibrv3L0', 'zTn-nNj5Bng', 'W6', 'F2Ny7rq9RKs', 'NVGGgncVq-4', '30PpKtVT8Z4', 'xxHx6s', 'Xp6rxW4hFFo', 'a28s', 'fvBs0xpEZhQ', '0lh', '1rECrfrdNTo', 'fnpp8v9NbmY', 'jI58q6rcNLc', 'gvVsgOK1iJw', 'yBK02O1Oewc', 'swKrHEeOj9c', '3zrVqwV29V8', '4cgzdXlJksU', 'ItFqogTmAvQ', '4Tn5S48sCwg', 'SzEbtbNSg04', 'CBD9h0jUq3w', 'lc9bA-hvqHU', 'MPuKVz3sjBA', 'BVilbVCo9sU', 'xlPyHMkpaQY', '3OPQqH3YlHA']


In [ ]:
import os

def count_video_names_from_frames():
    # مسار مجلد الفريمات المؤقتة
    frames_folder_path = '/content/drive/My Drive/dataset/temporary_images'

    # استخراج أسماء مقاطع الفيديو من اسماء الملفات داخل المجلد
    video_names = [filename.split('_')[0] for filename in os.listdir(frames_folder_path)]

    # عدد أسماء مقاطع الفيديو
    video_names_count = len(video_names)

    return video_names_count

# استدعاء الدالة لعدد أسماء مقاطع الفيديو
video_names_count = count_video_names_from_frames()

# طباعة عدد أسماء مقاطع الفيديو
print("Number of video names extracted from frames:", video_names_count)


Number of video names extracted from frames: 48


In [ ]:
# تحديد أسماء مقاطع الفيديو المطلوبة
video_names = [
    'q5ZRMvjzhXQ_15_29', 'KqKUXRHHkCc_101_105', 's-XjRDsYuzU_0_12', 'lexLAjh8fPA_27_31', 'JYNzY9jmsQQ_2_10',
    'HZ-BuDDmvVk_0_10', 'AzMrltkXGJk_0_10', '5E66Gk3V1Bc_23_30', '6FmN22YcVVE_11_18', 'ACOmKiJDkA4_47_54',
    '6gQu8PWhFoQ_37_41', 'ptHkvYrH9fY_2_10', 'ao-9B8IV9_E_108_119', 'R24RktgUi-s_24_28', 'HJHPB_Y8GZE_22_28',
    'QsC8__3lwO0_50_60', 'r4qv_BNlQNk_119_124', 'Pr5LQq42l4M_12_17', 'n_Z0-giaspE_437_447', 'Qjbcibrv3L0_26_32',
    'zTn-nNj5Bng_8_19', 'W6_XuNhgtrM_2_5', 'F2Ny7rq9RKs_139_148', 'NVGGgncVq-4_12_20', '3OPQqH3YlHA_50_54',
    'xxHx6s_DbUo_49_56', 'Xp6rxW4hFFo_32_40', 'a28s_wyqkyc_68_77', '0lh_UWF9ZP4_191_197', '1rECrfrdNTo_200_207',
    'fnpp8v9NbmY_181_188', 'jI58q6rcNLc_8_18', 'gvVsgOK1iJw_32_38', 'yBK02O1Oewc_20_35', 'swKrHEeOj9c_4_8',
    '3zrVqwV29V8_69_76', '4cgzdXlJksU_83_90', 'ItFqogTmAvQ_361_365', '4Tn5S48sCwg_110_118', 'SzEbtbNSg04_70_85',
    'CBD9h0jUq3w_1_7', 'lc9bA-hvqHU_1_6', 'MPuKVz3sjBA_1_12', 'BVilbVCo9sU_1_11', 'xlPyHMkpaQY_1_7', '3OPQqH3YlHA_50_54'
]

# قراءة ملف التوصيفات
with open('/content/MSVD_DATASET_CORPUS/annotations.txt', 'r') as file:
    lines = file.readlines()

# استخراج الأسماء المطلوبة
selected_videos = [line for line in lines if any(video in line for video in video_names)]

# كتابة الأسماء إلى ملف جديد
with open('/content/drive/MyDrive/dataset/selected_videos.txt', 'w') as output_file:
    output_file.writelines(selected_videos)


In [8]:
import os
import random
from keras.preprocessing.text import Tokenizer
import numpy as np

# ذكر نسبة تقسيم البيانات بين التدريب والاختبار
train_split = 0.85

def preprocessing():
    # تحديث مسارات الملفات
    TRAIN_LABEL_PATH = '/content/drive/MyDrive/dataset/selected_videos.txt'
    TRAIN_FEATURE_DIR = '/content/drive/MyDrive/dataset/feat'

    # قراءة ملف التوصيفات
    with open(TRAIN_LABEL_PATH, 'r') as data_file:
        lines = data_file.readlines()

    # قائمة التوصيفات والمفردات
    train_list = []
    vocab_list = []

    for line in lines:
        # نفترض أن اسم الفيديو والتوصيف مفصولان بمسافة
        video_id, caption = line.strip().split(' ', 1)
        caption = "<bos> " + caption + " <eos>"

        # استخدام الجمل التي يكون طولها بين 6 و 10 كلمات فقط
        if len(caption.split()) > 10 or len(caption.split()) < 6:
            continue
        else:
            train_list.append([caption, video_id])

    random.shuffle(train_list)
    training_list = train_list[:int(len(train_list) * train_split)]
    validation_list = train_list[int(len(train_list) * train_split):]

    for train in training_list:
        vocab_list.append(train[0])




    # إعداد التوكنات
    tokenizer = Tokenizer(num_words=1500)
    tokenizer.fit_on_texts(vocab_list)

    x_data = {}

    # تحميل جميع ملفات numpy arrays وحفظها في قاموس
    for filename in os.listdir(TRAIN_FEATURE_DIR):
        if filename.endswith('.npy'):
            f = np.load(os.path.join(TRAIN_FEATURE_DIR, filename))
            x_data[filename[:-4]] = f

    return training_list, validation_list, x_data, tokenizer

# استدعاء دالة المعالجة المسبقة
training_list, validation_list, x_data, tokenizer = preprocessing()


In [9]:
training_list, validation_list, x_data, tokenizer = preprocessing()

In [10]:
training_list

[['<bos> a girl cutting paper with a scissor <eos>', '4cgzdXlJksU_83_90'],
 ['<bos> someone is mixing dough <eos>', 'fnpp8v9NbmY_181_188'],
 ['<bos> a person is buried in the sand <eos>', 'gvVsgOK1iJw_32_38'],
 ['<bos> the woman is seasoning eggs <eos>', 'F2Ny7rq9RKs_139_148'],
 ['<bos> dog in the forest <eos>', 'JYNzY9jmsQQ_2_10'],
 ['<bos> someone is riding a horse <eos>', 'zTn-nNj5Bng_8_19'],
 ['<bos> a lady is jumping from the building <eos>', 'Pr5LQq42l4M_12_17'],
 ['<bos> a dad is talking to his kids <eos>', 'gvVsgOK1iJw_32_38'],
 ['<bos> a girl is playing a flute <eos>', '6FmN22YcVVE_11_18'],
 ['<bos> a woman rides a horse <eos>', 'zTn-nNj5Bng_8_19'],
 ['<bos> a baby being spoon fed <eos>', 'q5ZRMvjzhXQ_15_29'],
 ['<bos> people are playing soccer on the beach <eos>', 'CBD9h0jUq3w_1_7'],
 ['<bos> a man ios riding a horse <eos>', 'zTn-nNj5Bng_8_19'],
 ['<bos> a man is under the sand <eos>', 'gvVsgOK1iJw_32_38'],
 ['<bos> a man is running on a soccer field <eos>', 'R24RktgUi-s_24_2

In [11]:
import pickle

# حفظ الـ tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# حفظ x_data
with open('x_data.pickle', 'wb') as handle:
    pickle.dump(x_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [12]:
import pickle


# مسار المجلد في Google Drive حيث تريد حفظ الملفات
drive_path = '/content/drive/MyDrive/dataset/'

# حفظ الـ tokenizer
with open(drive_path + 'tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# حفظ x_data
with open(drive_path + 'x_data.pickle', 'wb') as handle:
    pickle.dump(x_data, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [13]:
# Load tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load x_data
with open('x_data.pickle', 'rb') as handle:
    x_data = pickle.load(handle)


In [14]:

import keras
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import joblib

time_steps_encoder=80
num_encoder_tokens=4096
latent_dim=512
time_steps_decoder=80
num_decoder_tokens=1500
batch_size=320
max_probability = -1

encoder_inputs = Input(shape=(time_steps_encoder, num_encoder_tokens), name="encoder_inputs")
encoder = LSTM(latent_dim, return_state=True,return_sequences=True, name='endcoder_lstm')
_, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


# Set up the decoder
decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name= "decoder_inputs")
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_relu')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, 80, 4096)]           0         []                            
 )                                                                                                
                                                                                                  
 decoder_inputs (InputLayer  [(None, 80, 1500)]           0         []                            
 )                                                                                                
                                                                                                  
 endcoder_lstm (LSTM)        [(None, 80, 512),            9439232   ['encoder_inputs[0][0]']      
                              (None, 512),                                                    

In [15]:
def load_datatest(train_path, epochs=100, x_data=x_data, tokenizer=tokenizer, num_decoder_tokens=1500,training_list=training_list, batch_size=32, maxlen=10):
    encoder_input_data = []
    decoder_input_data = []
    decoder_target_data = []
    videoId = []
    videoSeq = []
    # separating the videoId and the video captions
    for idx, cap in enumerate(training_list):
        caption = cap[0]
        videoId.append(cap[1])
        videoSeq.append(caption)
    # converting the captions to tokens and padding them to equal sizes
    train_sequences = tokenizer.texts_to_sequences(videoSeq)
    train_sequences = pad_sequences(train_sequences, padding='post', truncating='post', maxlen=maxlen)
    train_sequences = np.array(train_sequences)
    max_seq_length = train_sequences.shape[1]
    filesize = len(train_sequences)
    X_data = []
    y_data = []
    vCount = 0
    n = 0
    for i in range(epochs):
      for idx in  range(0,filesize):
        n += 1
        encoder_input_data.append(x_data[videoId[idx]])
        y = to_categorical(train_sequences[idx], num_decoder_tokens)
        decoder_input_data.append(y[:-1])
        decoder_target_data.append(y[1:])
        if n == batch_size:
          encoder_input = np.array(encoder_input_data)
          decoder_input = np.array(decoder_input_data)
          decoder_target = np.array(decoder_target_data)
          encoder_input_data = []
          decoder_input_data = []
          decoder_target_data = []
          n = 0
          yield ([encoder_input, decoder_input], decoder_target)

In [16]:
train = load_datatest(training_list)
valid = load_datatest(validation_list)

In [17]:
train_path = "/content/drive/MyDrive/dataset"

batch_size = 7
learning_rate = 0.0007
epochs = 150
latent_dim = 512
num_encoder_tokens = 4096
num_decoder_tokens = 1500
time_steps_encoder = 80
max_probability = -1
save_model_path = 'model_final'
validation_split = 0.15
max_length = 10
search_type = 'greedy'

In [18]:
# Run training
opt = keras.optimizers.Adam(learning_rate=0.0003)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.1, patience=5, verbose=0,mode="auto")
model.compile(metrics=['accuracy'], optimizer=opt, loss='categorical_crossentropy')

In [19]:
steps_per_epoch = len(training_list) // batch_size
validation_steps = len(validation_list) // batch_size


In [20]:
steps_per_epoch

174

In [21]:
validation_steps

30

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='min')

In [23]:
model.fit(train, validation_data=valid, validation_steps=validation_steps,
                  epochs=epochs, steps_per_epoch=steps_per_epoch,
                  callbacks=[reduce_lr, early_stopping])

Epoch 1/150
174/174 [==============================] - 15s 58ms/step - loss: 3.2611 - accuracy: 0.4029 - val_loss: 2.5090 - val_accuracy: 0.4856 - lr: 3.0000e-04
Epoch 2/150
174/174 [==============================] - 9s 54ms/step - loss: 2.2833 - accuracy: 0.5063 - val_loss: 2.0914 - val_accuracy: 0.5240 - lr: 3.0000e-04
Epoch 3/150
174/174 [==============================] - 12s 67ms/step - loss: 1.9695 - accuracy: 0.5462 - val_loss: 1.8772 - val_accuracy: 0.5478 - lr: 3.0000e-04
Epoch 4/150
174/174 [==============================] - 10s 55ms/step - loss: 1.7635 - accuracy: 0.5903 - val_loss: 1.6266 - val_accuracy: 0.6178 - lr: 3.0000e-04
Epoch 5/150
174/174 [==============================] - 9s 50ms/step - loss: 1.5850 - accuracy: 0.6175 - val_loss: 1.4967 - val_accuracy: 0.6301 - lr: 3.0000e-04
Epoch 6/150
174/174 [==============================] - 10s 55ms/step - loss: 1.4646 - accuracy: 0.6332 - val_loss: 1.3810 - val_accuracy: 0.6429 - lr: 3.0000e-04
Epoch 7/150
174/174 [=========

174/174 [==============================] - 8s 45ms/step - loss: 0.7336 - accuracy: 0.7873 - val_loss: 0.7093 - val_accuracy: 0.7894 - lr: 3.0000e-04


In [24]:
if not os.path.exists(save_model_path):
  os.makedirs(save_model_path)

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
[decoder_inputs] + decoder_states_inputs,
[decoder_outputs] + decoder_states)


In [25]:
encoder_model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_inputs (InputLayer  [(None, 80, 4096)]        0         
 )                                                               
                                                                 
 endcoder_lstm (LSTM)        [(None, 80, 512),         9439232   
                              (None, 512),                       
                              (None, 512)]                       
                                                                 
Total params: 9439232 (36.01 MB)
Trainable params: 9439232 (36.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
decoder_model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 decoder_inputs (InputLayer  [(None, 80, 1500)]           0         []                            
 )                                                                                                
                                                                                                  
 input_2 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 decoder_lstm (LSTM)         [(None, 80, 512),            4122624   ['decoder_inputs[0][0]',

In [27]:
# saving the models
encoder_model.save(os.path.join(save_model_path, 'encoder_model.h5'))
decoder_model.save_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))
with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'wb') as file:
  joblib.dump(tokenizer, file)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
# saving the models
encoder_model.save(os.path.join(save_model_path, 'encoder_model.keras'))
decoder_model.save_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))
with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'wb') as file:
  joblib.dump(tokenizer, file)


In [29]:
model.save(os.path.join(save_model_path, 'model.h5'))


In [30]:
decoder_model.save(os.path.join(save_model_path, 'decoder_model.keras'))


In [31]:
decoder_model.save(os.path.join(save_model_path, 'decoder_model.h5'))


In [38]:
import os
import shutil



# مسار المجلد الذي تريد حفظه في Google Drive
save_model_path = 'model_final'
drive_save_path = '/content/drive/My Drive/model_final'

# نسخ المجلد إلى Google Drive
shutil.copytree(save_model_path, drive_save_path)

print(f"The model has been saved to {drive_save_path}")


The model has been saved to /content/drive/My Drive/model_final


In [39]:
!ls /content/drive/My\ Drive/model_final


decoder_model.h5     decoder_model_weights.h5  encoder_model.keras  tokenizer1500
decoder_model.keras  encoder_model.h5	       model.h5


In [40]:
MSVD="/content/drive/MyDrive/MSVD_1"

In [41]:
import numpy as np
import cv2
import os
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.models import Model
from google.colab import drive
import shutil

# ربط Google Drive بجلسة Colab
drive.mount('/content/drive')

# تعديل config مباشرة هنا بدلاً من ملف خارجي
class config:
    test_path = '/content/drive/My Drive'  # مسار مقاطع الفيديو على Google Drive
    test_path1 = '/content/drive/My Drive/set'  # مسار الميزات والإطارات المستخرجة على Google Drive

def video_to_frames(video):
    video_id = video.split(".")[0]  # اسم مقطع الفيديو
    path = os.path.join(config.test_path1, 'temporary', video_id)  # المسار لمجلد الفريمات لهذا المقطع

    # إذا كانت الفريمات موجودة، قم بإرجاعها بدون إعادة الاستخراج
    if os.path.exists(path) and len(os.listdir(path)) > 0:
        image_list = [os.path.join(path, img) for img in sorted(os.listdir(path)) if img.endswith('.jpg')]
        return image_list

    # إذا لم تكن الفريمات موجودة، قم باستخراجها
    os.makedirs(path, exist_ok=True)
    video_path = os.path.join(config.test_path, 'MSVD_1', video)
    count = 0
    image_list = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if ret is False:
            break
        cv2.imwrite(os.path.join(path, f'frame{count}.jpg'), frame)  # حفظ الفريم في مجلد المقطع
        image_list.append(os.path.join(path, f'frame{count}.jpg'))
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return image_list

def model_cnn_load():
    model = ResNet152(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final

def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img

def extract_features(video, model):
    video_id = video.split(".")[0]
    print(video_id)
    print(f'Processing video {video}')

    image_list = video_to_frames(video)
    samples = np.round(np.linspace(0, len(image_list) - 1, 80))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    return img_feats

def extract_feats_pretrained_cnn():
    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(config.test_path1, 'feat1')):
        os.makedirs(os.path.join(config.test_path1, 'feat1'))

    video_list = os.listdir(os.path.join(config.test_path, 'MSVD_1'))

    if '.ipynb_checkpoints' in video_list:
        video_list.remove('.ipynb_checkpoints')

    for video in video_list:
        video_id = video.split(".")[0]
        outfile = os.path.join(config.test_path1, 'feat1', video_id + '.npy')

        # إذا كانت الميزات موجودة، تخطي هذا الفيديو
        if os.path.exists(outfile):
            print(f"Features for video {video} already exist. Skipping extraction.")
            continue

        # إذا لم تكن الميزات موجودة، استخراج الميزات والفريمات
        img_feats = extract_features(video, model)
        np.save(outfile, img_feats)
        print(f"Features for video {video} have been extracted and saved.")

# لتشغيل الكود مباشرة لاستخراج الميزات لجميع مقاطع الفيديو
extract_feats_pretrained_cnn()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
242900224/242900224 [==============================] - 9s 0us/step
Model loaded
set
Processing video set


IndexError: list index out of range

In [42]:
test_path="/content/drive/MyDrive/dataset/"

In [43]:
# تحديد أسماء مقاطع الفيديو المطلوبة
video_names = [
    '_O9kWD8nuRU_45_49', '_O9kWD8nuRU_37_43', '-_hbPLsZvvo_288_305', '-_hbPLsZvvo_269_275'
]

# قراءة ملف التوصيفات
with open('/content/MSVD_DATASET_CORPUS/annotations.txt', 'r') as file:
    lines = file.readlines()

# استخراج الأسماء المطلوبة
selected_videos = [line for line in lines if any(video in line for video in video_names)]

# كتابة الأسماء إلى ملف جديد
with open('/content/drive/MyDrive/dataset/selected_videos1.txt', 'w') as output_file:
    output_file.writelines(selected_videos)




In [45]:
from google.colab import drive
import shutil

# ربط كولاب بحساب Google Drive الخاص بك
drive.mount('/content/drive')

# تحديد مسار المجلدات
source_folder = '/content/drive/MyDrive/set'
destination_folder = '/content/drive/MyDrive/MSVD_1'

# نقل الملفات من مجلد1 إلى مجلد2
shutil.move(source_folder, destination_folder)

# إعادة تحميل المحرك للتأكد من التحديثات
drive.flush_and_unmount()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import functools
import operator
import os
import time

import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import joblib
import tensorflow as tf

search_type = 'beam'
save_model_path = '/content/drive/MyDrive/model_final'  # Adjust this to your actual save model path
test_path="/content/drive/MyDrive/" # Adjust this to your actual test path
num_decoder_tokens = 1500
latent_dim = 512

class VideoDescriber:
    def __init__(self):
        # تحميل النماذج والمعجم
        with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'rb') as file:
            self.tokenizer = joblib.load(file)
        self.inf_encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.h5'))
        decoder_inputs = Input(shape=(None, num_decoder_tokens))
        decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        self.inf_decoder_model.load_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

    def index_to_word(self):
        return {value: key for key, value in self.tokenizer.word_index.items()}

    def greedy_search(self, loaded_array):
        inv_map = self.index_to_word()
        states_value = self.inf_encoder_model.predict(loaded_array.reshape(-1, 80, 4096))
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

    def decode_sequence2bs(self, input_seq):
        states_value = self.inf_encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value, [], [], 0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob, path, lens):
        global max_probability, decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
        output_tokens = output_tokens.reshape(num_decoder_tokens)
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 12
            if sampled_char != 'eos' and lens <= MAX_LEN:
                p = output_tokens[sampled_token_index[i]]
                if sampled_char == '':
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)

                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens + 1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if p > max_probability:
                    decode_seq = path
                    max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        uni_gram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in uni_gram:
                uni_gram[c] += 1
            else:
                uni_gram[c] = 1
            if last_string == c and idx2 > 0:
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self):
        X_test = []
        X_test_filename = []
        #/content/drive/MyDrive/testing_id.txt
        #test_path="/content/drive/MyDrive/"
        with open(os.path.join('/content/drive/MyDrive', 'testing_id.txt')) as testing_file:
            lines = testing_file.readlines()
            for filename in lines:
                filename = filename.strip()
                #/content/drive/MyDrive/MSVD_1/set/feat1     /content/drive/MyDrive/MSVD_1/set
                file_path = "/content/drive/MyDrive/MSVD_1/set/feat1/-_hbPLsZvvo_269_275.npy"
                X_test.append(file_path)
                X_test_filename.append(filename[:-4])
            X_test = np.array(X_test)
        return X_test, X_test_filename



    def test(self):
        global max_probability
        X_test, X_test_filename = self.get_test_data()
        for search_type in ['greedy', 'beam']:
            with open(os.path.join(test_path, f'test1_{search_type}.txt'), 'w') as file:
                for idx, x in enumerate(X_test):
                    file.write(X_test_filename[idx] + ',')
                    start = time.time()
                    #if search_type == 'greedy':
                        #decoded_sentence = self.greedy_search(x.reshape(-1, 80, 4096))
                    #else:
                    decoded_sentence = self.decode_sequence2bs(x.reshape(-1, 80, 4096))
                    decode_str = self.decoded_sentence_tuning(decoded_sentence)
                    decoded_sentence = ' '.join(decode_str)
                    file.write(decoded_sentence + ',{:.2f}'.format(time.time() - start))
                    file.write('\n')

                    max_probability = -1

if __name__ == "__main__":
    video_to_text = VideoDescriber()
    video_to_text.test()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/model_final/tokenizer1500'

In [49]:
import functools
import operator
import os
import time

import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import joblib
import tensorflow as tf

search_type = 'beam'
save_model_path = '/content/drive/MyDrive/model_final'  # Adjust this to your actual save model path
test_path = "/content/drive/MyDrive/"  # Adjust this to your actual test path
num_decoder_tokens = 1500
latent_dim = 512

class VideoDescriber:
    def __init__(self):
        # تحميل النماذج والمعجم
        with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'rb') as file:
            self.tokenizer = joblib.load(file)
        self.inf_encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.h5'))
        decoder_inputs = Input(shape=(None, num_decoder_tokens))
        decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        self.inf_decoder_model.load_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

    def index_to_word(self):
        return {value: key for key, value in self.tokenizer.word_index.items()}

    def greedy_search(self, loaded_array):
        inv_map = self.index_to_word()
        try:
            reshaped_array = loaded_array.reshape(-1, 80, 4096)
        except ValueError:
            print(f"Error reshaping array of size {loaded_array.size}")
            return ""
        states_value = self.inf_encoder_model.predict(reshaped_array)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

    def decode_sequence2bs(self, input_seq):
        try:
            reshaped_array = input_seq.reshape(-1, 80, 4096)
        except ValueError:
            print(f"Error reshaping array of size {input_seq.size}")
            return ""
        states_value = self.inf_encoder_model.predict(reshaped_array)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value, [], [], 0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob, path, lens):
        global max_probability, decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
        output_tokens = output_tokens.reshape(num_decoder_tokens)
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 12
            if sampled_char != 'eos' and lens <= MAX_LEN:
                p = output_tokens[sampled_token_index[i]]
                if sampled_char == '':
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)

                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens + 1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if p > max_probability:
                    decode_seq = path
                    max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        uni_gram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in uni_gram:
                uni_gram[c] += 1
            else:
                uni_gram[c] = 1
            if last_string == c and idx2 > 0:
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self):
        X_test = []
        X_test_filename = []
        with open(os.path.join('/content/drive/MyDrive', 'testing_id.txt')) as testing_file:
            lines = testing_file.readlines()
            for filename in lines:
                filename = filename.strip()
                file_path = "/content/drive/MyDrive/MSVD_1/set/feat1/" + filename
                if os.path.isfile(file_path):
                    data = np.load(file_path)
                    if data.size == 80 * 4096:  # Check if the size matches
                        X_test.append(data)
                        X_test_filename.append(filename[:-4])
                    else:
                        print(f"Skipping {filename} due to size mismatch")
            X_test = np.array(X_test)
        return X_test, X_test_filename

    def test(self):
        global max_probability
        X_test, X_test_filename = self.get_test_data()
        for search_type in ['greedy', 'beam']:
            with open(os.path.join(test_path, f'test_{search_type}.txt'), 'w') as file:
                for idx, x in enumerate(X_test):
                    file.write(X_test_filename[idx] + ',')
                    start = time.time()
                    if search_type == 'greedy':
                        decoded_sentence = self.greedy_search(x)
                    else:
                        decoded_sentence = self.decode_sequence2bs(x)
                    decode_str = self.decoded_sentence_tuning(decoded_sentence)
                    decoded_sentence = ' '.join(decode_str)
                    file.write(decoded_sentence + ',{:.2f}'.format(time.time() - start))
                    file.write('\n')

                    max_probability = -1

if __name__ == "__main__":
    video_to_text = VideoDescriber()
    video_to_text.test()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/model_final/tokenizer1500'

In [48]:
import functools
import operator
import os
import time

import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import joblib
import tensorflow as tf

search_type = 'beam'
save_model_path = '/content/drive/MyDrive/model_final'  # Adjust this to your actual save model path
test_path = "/content/drive/MyDrive/"  # Adjust this to your actual test path
num_decoder_tokens = 1500
latent_dim = 512

class VideoDescriber:
    def __init__(self):
        # تحميل النماذج والمعجم
        with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'rb') as file:
            self.tokenizer = joblib.load(file)

        self.inf_encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.h5'), compile=False)
        self.inf_encoder_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        decoder_inputs = Input(shape=(None, num_decoder_tokens))
        decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        self.inf_decoder_model.load_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

    def index_to_word(self):
        return {value: key for key, value in self.tokenizer.word_index.items()}

    def greedy_search(self, loaded_array):
        inv_map = self.index_to_word()
        try:
            reshaped_array = loaded_array.reshape(-1, 80, 4096)
        except ValueError:
            print(f"Error reshaping array of size {loaded_array.size}")
            return ""
        states_value = self.inf_encoder_model.predict(reshaped_array)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

    def decode_sequence2bs(self, input_seq):
        try:
            reshaped_array = input_seq.reshape(-1, 80, 4096)
        except ValueError:
            print(f"Error reshaping array of size {input_seq.size}")
            return ""
        states_value = self.inf_encoder_model.predict(reshaped_array)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value, [], [], 0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob, path, lens):
        global max_probability, decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
        output_tokens = output_tokens.reshape(num_decoder_tokens)
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 12
            if sampled_char != 'eos' and lens <= MAX_LEN:
                p = output_tokens[sampled_token_index[i]]
                if sampled_char == '':
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)

                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens + 1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if p > max_probability:
                    decode_seq = path
                    max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        uni_gram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in uni_gram:
                uni_gram[c] += 1
            else:
                uni_gram[c] = 1
            if last_string == c and idx2 > 0:
                continue
            if c in filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self):
        X_test = []
        X_test_filename = []
        with open(os.path.join('/content/drive/MyDrive', 'testing_id.txt')) as testing_file:
            lines = testing_file.readlines()
            for filename in lines:
                filename = filename.strip()
                file_path = "/content/drive/MyDrive/MSVD_1/set/feat1/" + filename
                if os.path.isfile(file_path):
                    data = np.load(file_path)
                    if data.size == 80 * 4096:  # Check if the size matches
                        X_test.append(data)
                        X_test_filename.append(filename[:-4])
                    else:
                        print(f"Skipping {filename} due to size mismatch")
            X_test = np.array(X_test)
        return X_test, X_test_filename

    def test(self):
        global max_probability
        X_test, X_test_filename = self.get_test_data()
        for search_type in ['greedy', 'beam']:
            with open(os.path.join(test_path, f'test1_{search_type}.txt'), 'w') as file:
                for idx, x in enumerate(X_test):
                    file.write(X_test_filename[idx] + ',')
                    start = time.time()
                    if search_type == 'greedy':
                        decoded_sentence = self.greedy_search(x)
                    else:
                        decoded_sentence = self.decode_sequence2bs(x)
                    decode_str = self.decoded_sentence_tuning(decoded_sentence)
                    decoded_sentence = ' '.join(decode_str)
                    file.write(decoded_sentence + ',{:.2f}'.format(time.time() - start))
                    file.write('\n')

                    max_probability = -1

if __name__ == "__main__":
    video_to_text = VideoDescriber()
    video_to_text.test()


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/model_final/tokenizer1500'

In [47]:
import functools
import operator
import os
import time

import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model, load_model
import joblib
import tensorflow as tf

search_type = 'beam'
save_model_path = '/content/drive/MyDrive/model_final'  # Adjust this to your actual save model path
test_path = "/content/drive/MyDrive/"  # Adjust this to your actual test path
num_decoder_tokens = 1500
latent_dim = 512

class VideoDescriber:
    def __init__(self):
        # تحميل النماذج والمعجم
        with open(os.path.join(save_model_path, 'tokenizer' + str(num_decoder_tokens)), 'rb') as file:
            self.tokenizer = joblib.load(file)
        self.inf_encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.h5'))
        decoder_inputs = Input(shape=(None, num_decoder_tokens))
        decoder_dense = Dense(num_decoder_tokens, activation='softmax')
        decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = decoder_dense(decoder_outputs)
        self.inf_decoder_model = Model(
            [decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)
        self.inf_decoder_model.load_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

    def index_to_word(self):
        return {value: key for key, value in self.tokenizer.word_index.items()}

    def greedy_search(self, loaded_array):
        inv_map = self.index_to_word()
        states_value = self.inf_encoder_model.predict(loaded_array.reshape(-1, 80, 4096))
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        sentence = ''
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        for i in range(15):
            output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
            states_value = [h, c]
            output_tokens = output_tokens.reshape(num_decoder_tokens)
            y_hat = np.argmax(output_tokens)
            if y_hat == 0:
                continue
            if inv_map[y_hat] is None:
                break
            else:
                sentence = sentence + inv_map[y_hat] + ' '
                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, y_hat] = 1
        return ' '.join(sentence.split()[:-1])

    def decode_sequence2bs(self, input_seq):
        states_value = self.inf_encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
        self.beam_search(target_seq, states_value, [], [], 0)
        return decode_seq

    def beam_search(self, target_seq, states_value, prob, path, lens):
        global max_probability, decode_seq
        node = 2
        output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
        output_tokens = output_tokens.reshape(num_decoder_tokens)
        sampled_token_index = output_tokens.argsort()[-node:][::-1]
        states_value = [h, c]
        for i in range(node):
            if sampled_token_index[i] == 0:
                sampled_char = ''
            else:
                sampled_char = list(self.tokenizer.word_index.keys())[list(self.tokenizer.word_index.values()).index(sampled_token_index[i])]
            MAX_LEN = 12
            if sampled_char != 'eos' and lens <= MAX_LEN:
                p = output_tokens[sampled_token_index[i]]
                if sampled_char == '':
                    p = 1
                prob_new = list(prob)
                prob_new.append(p)
                path_new = list(path)
                path_new.append(sampled_char)

                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, sampled_token_index[i]] = 1.
                self.beam_search(target_seq, states_value, prob_new, path_new, lens + 1)
            else:
                p = output_tokens[sampled_token_index[i]]
                prob_new = list(prob)
                prob_new.append(p)
                p = functools.reduce(operator.mul, prob_new, 1)
                if p > max_probability:
                    decode_seq = path
                    max_probability = p

    def decoded_sentence_tuning(self, decoded_sentence):
        decode_str = []
        filter_string = ['bos', 'eos']
        uni_gram = {}
        last_string = ""
        for idx2, c in enumerate(decoded_sentence):
            if c in uni_gram:
                uni_gram[c] += 1
            else:
                uni_gram[c] = 1
            if last_string == c and idx2 > 0:
                continue
            if c في filter_string:
                continue
            if len(c) > 0:
                decode_str.append(c)
            if idx2 > 0:
                last_string = c
        return decode_str

    def get_test_data(self):
        X_test = []
        X_test_filename = []
        with open(os.path.join('/content/drive/MyDrive', 'testing_id.txt')) as testing_file:
            lines = testing_file.readlines()
            for filename in lines:
                filename = filename.strip()
                file_path = f"/content/drive/MyDrive/MSVD_1/set/feat1/{filename}"
                if os.path.isfile(file_path):
                    data = np.load(file_path)
                    X_test.append(data)
                    X_test_filename.append(filename[:-4])
            X_test = np.array(X_test)
        return X_test, X_test_filename

    def test(self):
        global max_probability
        X_test, X_test_filename = self.get_test_data()
        for search_type in ['greedy', 'beam']:
            with open(os.path.join(test_path, f'test1_{search_type}.txt'), 'w') as file:
                for idx, x in enumerate(X_test):
                    file.write(X_test_filename[idx] + ',')
                    start = time.time()
                    if search_type == 'greedy':
                        decoded_sentence = self.greedy_search(x.reshape(-1, 80, 4096))
                    else:
                        decoded_sentence = self.decode_sequence2bs(x.reshape(-1, 80, 4096))
                    decode_str = self.decoded_sentence_tuning(decoded_sentence)
                    decoded_sentence = ' '.join(decode_str)
                    file.write(decoded_sentence + ',{:.2f}'.format(time.time() - start))
                    file.write('\n')

                    max_probability = -1

if __name__ == "__main__":
    video_to_text = VideoDescriber()
    video_to_text.test()


SyntaxError: invalid syntax (<ipython-input-47-904322ce2145>, line 115)